In [1]:
"""
Let's Check Zero one prediction problem


"""

"\nLet's Check Zero one prediction problem\n\n\n"

In [2]:

from IPython.display import display, Markdown, Latex
h1 = lambda x: display(Markdown('# '+str(x)))
h3 = lambda x: display(Markdown('### '+str(x)))
h4 = lambda x: display(Markdown('#### '+ (str(x).replace(' ','&nbsp;') if str(x) else 'None')))
h5 = lambda x: display(Markdown('##### '+ (str(x).replace(' ','&nbsp;') if str(x) else 'None')))
bl = lambda x: display(Markdown('##### <font color="blue"> %s </font>'%(str(x).replace(' ','&nbsp;') if str(x) else 'None')))
rd = lambda x: display(Markdown('##### <font color="red"> %s </font>'%(str(x).replace(' ','&nbsp;') if str(x) else 'None')))
rd1 = lambda x: display(Markdown('# <font color="red"> %s </font>'%(str(x).replace(' ','&nbsp;') if str(x) else 'None')))
gra = lambda x: display(Markdown('##### <font color="gray"> %s </font>'%(str(x).replace(' ','&nbsp;') if str(x) else 'None')))
itc = lambda x: display(Markdown('***'+str(x.strip()) + '***' ))
pl = lambda: print('\n'+'#'*100 + '\n')
pl2 = lambda: print('\n'+'='*100 + '\n')
pls = lambda: print('\n'+'#'*50 +' Start '+'#'*50 + '\n')
pn = lambda x=5: print('\n'*x)

from collections import Counter


## Hand Made Neural Network

https://144.34.140.210:8579/notebooks/Git_Qiliang/ML_ANN_Gradiant_Decent/Backpropagation.ipynb

In [3]:
import pdb
bp = pdb.set_trace

import random
import uuid
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math


In [4]:
# Direct
def relu(x):
    return x
activation_func = np.vectorize(relu)  

         
def derivative(x):
    return 1
derivative_func = np.vectorize(derivative)   


# # Relu
# def relu(x):
#     if x >0:
#         return x
#     else:
#         return 0

# activation_func = np.vectorize(relu)  

         
# def derivative(x):
#     if x>0:
#         return 1
#     else:
#         return 0
# derivative_func = np.vectorize(derivative)   


# # Sigmoid
# def relu(x):
#     if x<-100:
#         return 0
#     elif x>100:
#         return 1
#     else:
#         y = 1/(1+np.exp(-x))
#         return y
# activation_func = np.vectorize(relu)  

         
# def derivative(x):
#     return x*(1-x)
# derivative_func = np.vectorize(derivative)   
    

    

# def relu(x):
#     if x>0:
#         if x<5:
#             return x
#         else:
#             return 5
#     else:
#         return 0


# activation_func = np.vectorize(relu)  
# # activation_func = np.vectorize(lambda x: x) 
         
# def derivative(x):
#     if x>0:
#             return 1
#     else:
#         return 0
# #         return -1
# #         return 1
# derivative_func = np.vectorize(derivative)   
    
    
class Weight:
    def __init__(self,m,n):
        self.sigma = 0.2# 0.1 # 0.3 #0.2
        self.mu = 0 #0.5
        self.m = m
        self.n = n
        self.val = self.sigma * np.random.randn(m,n) + self.mu
#         self.normalize()
        
    def normalize(self):
        self.val = self.val/self.val.sum()

class Layer:
    def __init__(self,n,id=None,output_ind=False):
        self.val = None
        self.id = id
        self.input_weight = None
        self.output_weight = None
        self.pre_layer = None
        self.next_layer = None
        self.n = n
        self.delta_weight = None
        self.output_ind = output_ind
        
        
    def full_connect(self,other):
        m = self.n
        n = other.n
        weight = Weight(m,n)
        self.output_weight = weight
        other.input_weight = weight
        
        self.next_layer = other
        other.pre_layer = self
    
    def stats(self):
        print("n:",self.n)
        if self.output_weight:
            print('Weight:',self.output_weight.val.shape)
        print()
            
    def pull_in(self,input_list):
        self.val = np.array([input_list,])
#         self.val = self.val/self.val.sum()
            
    def forwarding(self):
#         print(self.id,'forwarding')
        val = self.val.dot(self.output_weight.val)
        try:
            if self.next_layer:
                if not self.next_layer.output_ind:
                    self.next_layer.val = activation_func(val)   
                else:
                    self.next_layer.val = val 
        except:
#             bp()
            raise
        
    def backwarding(self, error_term, learning_rate):
#         print(self.id,'backwarding')
        pre = self.pre_layer
        K1 = pre.output_weight.val
#         print(K1.shape)
        while(pre.pre_layer):
            pre = pre.pre_layer
            K1 = pre.output_weight.val.dot(K1)
            
        K1 = pre.val.dot(K1)
#         print('K1:',K1.shape)


        next_layer = self.next_layer
        if next_layer.output_weight:
            K2 = next_layer.output_weight.val
            while(next_layer.next_layer.output_weight):
                next_layer = next_layer.next_layer
                K2 = K2.dot(next_layer.output_weight.val) 
#             print("K2:",K2.shape)
        else:
            K2 = np.array([[1]])
                   
        derivative = (K1.T*self.val.T).dot(K2.T)
#         derivative = K1.T.dot(K2.T)
#         bp()
    
        if not self.output_ind:
            K3 = derivative_func(self.val)
#             bp()
            derivative = derivative*K3.T
#         bp()
#         print("Derivative:",derivative.shape)
#         return derivative

#         derivative = derivative/derivative.sum()
    
        delta_weight = learning_rate*(-1)*(error_term)*derivative
#         bp()
#         print('delta_weight:',delta_weight)
        if type(self.delta_weight)!=type(None):
            self.delta_weight += delta_weight
        else:
            self.delta_weight = delta_weight
    
    def update_weights(self):
        if type(self.delta_weight)!=type(None):
            self.output_weight.val += self.delta_weight
            self.delta_weight = None
    
class Network:
    def __init__(self,input_layer,output_layer):
        self.input_layer = input_layer
        self.output_layer = output_layer
        self.learning_rate = 0.01
        
    def set_learning_rate(self,rate):
        self.learning_rate = rate
        
              
    def batch_forwarding(self):
        cur = self.input_layer
        while(cur.next_layer):
            cur.forwarding()
            cur = cur.next_layer
             
    def get_error_term(self,ground_truth_list):      
        error = self.output_layer.val-ground_truth_list
        return error
    
    def batch_backwarding(self, error_term):
        cur = self.input_layer.next_layer
        while(cur and cur.output_weight):
            cur.backwarding(error_term, self.learning_rate)
            cur = cur.next_layer
            
    def one_train(self,x,y):
        input_layer.pull_in(x)
        self.batch_forwarding()
        error_term = self.get_error_term(y)
        self.batch_backwarding(error_term)
        
        
    def batch_update_weights(self):
        cur = self.input_layer.next_layer
        while(cur and cur.output_weight):
            cur.update_weights() 
            cur = cur.next_layer

    def batch_train(self,X,Y,batch_size=20):
        i=0
        for i,(x,y) in enumerate(zip(X,Y)):
            self.one_train(x,y)
            if i%batch_size==0:
                self.batch_update_weights()
        self.batch_update_weights()
        
        
    def predict(self,x):
        input_layer.pull_in(x)
        self.batch_forwarding()
        return self.output_layer.val
    
    
#     def save(self,model_dir='./model.json'):
#         res = []
#         cur = self.input_layer
#         while(cur.output_weight):
#             res.append(cur)
         
#     def load(self,model_dir='./model.json')
#         pass



# # Networks
# input_layer = Layer(10,'in')
# l1 = Layer(10,'l1')
# l2 = Layer(50,'l2')
# l3 = Layer(50,'l3')
# l4 = Layer(10,'l4')
# output_layer = Layer(1,'out')


# input_layer.full_connect(l1)
# l1.full_connect(l2)
# l2.full_connect(l3)
# l3.full_connect(l4)
# l4.full_connect(output_layer)

# x = random.choices(range(100), k=10)

# print(x)
# y = np.array([[1]])

# nk = Network(input_layer,output_layer)
# nk.one_train(x,y)



In [5]:
"""
最简单和最直接的方法是什么

看起来不行

如何保证每次都有进步呢？  这种简单的信息是如何被学到的呢？

这是一个超级不稳定的系统


产生合理的顿感系数



"""

'\n最简单和最直接的方法是什么\n\n看起来不行\n\n如何保证每次都有进步呢？  这种简单的信息是如何被学到的呢？\n\n这是一个超级不稳定的系统\n\n\n产生合理的顿感系数\n\n\n\n'

In [70]:
# # Networks
# input_layer = Layer(10,'in')
# l1 = Layer(10,'l1')
# l2 = Layer(50,'l2')
# l3 = Layer(50,'l3')
# l4 = Layer(10,'l4')
# output_layer = Layer(1,'out',output_ind=True)

# input_layer.full_connect(l1)
# l1.full_connect(l2)
# l2.full_connect(l3)
# l3.full_connect(l4)
# l4.full_connect(output_layer)


# # Networks
# input_layer = Layer(10,'in')
# l1 = Layer(10,'l1')
# l2 = Layer(50,'l2')
# l2_5 = Layer(50,'l2.5')
# l3 = Layer(50,'l3')
# l4 = Layer(10,'l4')
# output_layer = Layer(1,'out',output_ind=True)

# input_layer.full_connect(l1)
# l1.full_connect(l2)
# l2.full_connect(l2_5)
# l2_5.full_connect(l3)
# l3.full_connect(l4)
# l4.full_connect(output_layer)

# # Networks
# input_layer = Layer(1,'in')
# l1 = Layer(10,'l1')
# l2 = Layer(50,'l2')
# l3 = Layer(50,'l3')
# l4 = Layer(50,'l4')
# output_layer = Layer(1,'out',output_ind=True)

# input_layer.full_connect(l1)
# l1.full_connect(l2)
# l2.full_connect(l3)
# l3.full_connect(l4)
# l4.full_connect(output_layer)



# Networks
input_layer = Layer(1,'in')
l1 = Layer(10,'l1')
l2 = Layer(10,'l2')
output_layer = Layer(1,'out',output_ind=True)

input_layer.full_connect(l1)
l1.full_connect(l2)
l2.full_connect(output_layer)

nk = Network(input_layer,output_layer)

In [85]:
X = []
Y = []
# for i in range(1):
# for i in range(10):
# for i in range(100):
# for i in range(500):
# for i in range(1000):
for i in range(10000):
# for i in range(100000):
# for i in range(1000000):
# for i in range(5000000):
# for i in range(15000000):
# for i in range(50000000):
#     x = np.array(random.choices(range(100), k=10))
    x = np.array([np.random.rand(),])
    y=x
    X.append(x)
    Y.append(y)

In [86]:
# nk.set_learning_rate(0.0000000000001)
# nk.set_learning_rate(0.00000001)
# nk.set_learning_rate(0.00001)
# nk.set_learning_rate(0.0001)
# nk.set_learning_rate(0.001)
nk.set_learning_rate(0.01)
# nk.set_learning_rate(0.1)
# nk.set_learning_rate(1)
# nk.set_learning_rate(2)
nk.batch_train(X,Y)

In [87]:
x = np.array([np.random.rand(),])
if x>0.5:
    y=1
else:
    y=0
y_predict = nk.predict(x)

print('x:',x)
print('y_real:',y)
print('y_predict:',y_predict)
print('rate:',x/y_predict)

# print('l1:\n',l1.val)
# print('l2:\n',l2.val)
# print('l3:\n',l3.val)
# print('l4:\n',l4.val)
# pl()

x: [0.88318657]
y_real: 1
y_predict: [[0.88316317]]
rate: [[1.00002649]]


In [89]:
Y

[array([0.83839055]),
 array([0.05452792]),
 array([0.06518043]),
 array([0.4975972]),
 array([0.22401634]),
 array([0.81681695]),
 array([0.20850591]),
 array([0.90363255]),
 array([0.63853398]),
 array([0.05265187]),
 array([0.90627535]),
 array([0.12109381]),
 array([0.69661404]),
 array([0.03147795]),
 array([0.99888555]),
 array([0.73192348]),
 array([0.03549357]),
 array([0.7481998]),
 array([0.88163514]),
 array([0.75125931]),
 array([0.62837537]),
 array([0.03836684]),
 array([0.05310221]),
 array([0.86548945]),
 array([0.43882975]),
 array([0.6692632]),
 array([0.93217058]),
 array([0.20525629]),
 array([0.39593535]),
 array([0.25996756]),
 array([0.17762958]),
 array([0.83762203]),
 array([0.79958045]),
 array([0.27087759]),
 array([0.96963537]),
 array([0.66189971]),
 array([0.91908846]),
 array([0.92883397]),
 array([0.95717545]),
 array([0.8401233]),
 array([0.98213779]),
 array([0.18369321]),
 array([0.24354294]),
 array([0.27795841]),
 array([0.37061586]),
 array([0.9437

In [81]:
"""
线性的收敛非常快的


尝试下非0、1，看是否会收敛   可以收敛

需要有偏置项？

使用更简单的网络试试


也许可以储备多种终端传导方式供选择   

有多种结合体？ 不同特性的    如何连接

似乎没有学到   但是成功产生收敛了   因为weight叠加加成的影响，学习率变得很重要

初始weight的选取似乎影响很大

"""

'\n尝试下非0、1，看是否会收敛   可以收敛\n\n需要有偏置项？\n\n使用更简单的网络试试\n\n\n也许可以储备多种终端传导方式供选择   \n\n有多种结合体？ 不同特性的    如何连接\n\n似乎没有学到   但是成功产生收敛了   因为weight叠加加成的影响，学习率变得很重要\n\n初始weight的选取似乎影响很大\n\n'

In [55]:
input_layer.val

In [56]:
l1.output_weight.val

array([[-0.2845584 ,  0.13673248,  0.10354401, -0.08510641, -0.01888229,
        -0.00747488, -0.28601074,  0.01336099,  0.12996374,  0.05062755],
       [-0.14989739,  0.2399658 ,  0.04395194, -0.41291077, -0.22467109,
         0.26471604, -0.22413492,  0.04494837, -0.16465957, -0.14943373],
       [-0.00857381,  0.24669755,  0.21864967,  0.16284484, -0.02197971,
        -0.06773018,  0.080148  , -0.248923  ,  0.10173167,  0.2653519 ],
       [ 0.22614567,  0.05138435, -0.08981352,  0.028295  , -0.16459404,
        -0.06363028,  0.04771592,  0.34801123,  0.04652074,  0.14497922],
       [ 0.01093582, -0.1464401 ,  0.27885357, -0.11632317,  0.11099353,
         0.54548069,  0.12179685,  0.01277431,  0.08911185,  0.06337721],
       [-0.10912654,  0.15822046,  0.24152122, -0.22598017, -0.07870978,
        -0.21090868,  0.11301604, -0.02047031,  0.11618698,  0.07873891],
       [ 0.43043878, -0.12836605,  0.05624292, -0.22122099,  0.06382959,
        -0.24004305, -0.1714452 ,  0.24576639

In [264]:
l1.val

array([[ 0.00605304,  0.25327553, -0.08898177, -0.01885014,  0.23504693,
        -0.13334051, -0.03651148, -0.09328587,  0.16352103, -0.10575931]])

In [265]:
l2.output_weight.val

array([[ 0.07710758],
       [ 0.04125989],
       [ 0.06167409],
       [-0.06410992],
       [-0.03072252],
       [-0.38623125],
       [ 0.32221714],
       [ 0.13707543],
       [-0.30072998],
       [-0.01125444]])

In [210]:
l2.val

array([[-0.00334314, -0.01888611, -0.00417058, -0.01973776, -0.00770093,
         0.01558963,  0.00069752,  0.03247754,  0.00117863,  0.0086492 ]])

In [211]:
l3.output_weight.val

NameError: name 'l3' is not defined

In [212]:
l3.val

NameError: name 'l3' is not defined

In [173]:
l4.output_weight.val

array([[-0.59472918],
       [-0.24441027],
       [ 0.62609755],
       [-0.10691708],
       [ 0.30003725],
       [ 0.18347305],
       [ 0.18243542],
       [ 0.16239599],
       [-0.22661432],
       [ 0.02297569],
       [ 0.69880413],
       [ 0.56031021],
       [ 0.08163068],
       [ 0.04369627],
       [ 0.45100922],
       [ 0.18293961],
       [-0.0116464 ],
       [-0.55731291],
       [ 0.2540942 ],
       [ 0.09632412],
       [ 0.50653305],
       [-0.28051569],
       [ 0.0069488 ],
       [ 0.10843931],
       [ 0.03860016],
       [ 0.47657341],
       [-0.08330561],
       [-0.17609592],
       [-0.05295108],
       [-0.25617296],
       [ 0.55676044],
       [-0.07676087],
       [-0.07558063],
       [-0.01314843],
       [-0.23466799],
       [ 0.41508251],
       [-0.25120104],
       [ 0.01402803],
       [-0.13615577],
       [ 0.01784295],
       [ 0.30223832],
       [-0.57344112],
       [ 0.36587298],
       [-0.06703676],
       [ 0.10811401],
       [ 0

In [174]:
l4.val

array([[ 1.95236050e-01,  9.57827551e-02,  1.54415396e-02,
         1.90973974e-01, -1.90742693e-01,  1.40109049e-01,
         1.30953365e-01,  1.74646112e-01,  5.09469716e-02,
        -6.48411898e-02, -3.11482187e-01,  1.79225692e-01,
        -1.63757917e-04, -1.08607787e-01,  1.28668819e-02,
         1.73872095e-01,  8.19001427e-03,  1.69637233e-01,
         7.29088121e-03, -3.03357257e-01, -4.05746328e-01,
         1.66482086e-01, -6.84090987e-02,  8.80046900e-02,
         1.04884739e-01,  1.17373835e-01,  1.73952463e-02,
        -7.02409628e-02, -1.74645586e-01, -1.15991539e-01,
        -3.60247616e-02, -1.36674438e-02,  3.14791959e-01,
         8.52311187e-02,  4.59278015e-02,  2.57992622e-02,
         4.82661233e-02,  3.51150368e-03, -6.80787839e-02,
         1.16164178e-01, -4.16179338e-01,  2.80416144e-03,
        -3.33978546e-01,  4.34409281e-02, -2.09042447e-02,
        -2.25706641e-01, -3.88800903e-02,  9.62131200e-02,
         1.38550807e-01,  2.41214260e-01]])

In [175]:
output_layer.val

array([[-0.90226704]])

In [176]:
output_layer.val[0][0]

-0.9022670437452267

"""

当前设置非常容易出现参数消失


找一种函数可以自动的大变小，小变大

"""

## Notes

https://app.yinxiang.com/shard/s18/nl/18934792/e81511f4-0b23-48bc-b627-22416bbf9554

In [ ]:
"""


"""